<a href="https://colab.research.google.com/github/Pedro2318/Introduction-to-Data-Science-in-Python-assignments/blob/main/assignment2_Plotting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false; // disable scroll bar when displaying Folium map
}

<IPython.core.display.Javascript object>

# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Global Historical Climatology Network daily (GHCNd)](https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe - it's a wonderfully large dataset to play with! In particular, you will be asked to use data from the Ann Arbor Michigan location (my home!). and this is stored in the file: `assets/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`

Each row in this datafile corresponds to a single observation from a weather station, and has the following variables:
* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write a python notebook which plots line graphs of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015. (Based on the graph, do you think extreme weather is getting more frequent in 2015?)
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

I've written some steps I think would be good to go through, but there are other ways to solve this assignment so feel free to explore the pandas library! What I really want to see is an image that looks like this sketch I drew at my desk:

![](assets/chris_sketch.png)

In [ ]:
#  I'll be using the folium package to render the data into a map in Jupyter.

import folium
import pandas as pd

# get the location information for this dataset
df = pd.read_csv('assets/BinSize_d400.csv')
station_locations_by_hash = df[df['hash'] == 'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89']

# get longitude and lattitude to plot
lons = station_locations_by_hash['LONGITUDE'].tolist()
lats = station_locations_by_hash['LATITUDE'].tolist()

# plot on a beautiful folium map
my_map = folium.Map(location = [lats[0], lons[0]], height = 500,  zoom_start = 9)
for lat, lon in zip(lats, lons):
    folium.Marker([lat, lon]).add_to(my_map)

# render map in Jupyter
display(my_map)

FileNotFoundError: ignored

## Step 1
Load the dataset and transform the data into Celsius (refer to documentation) then extract all of the rows which have minimum or maximum temperatures.

__hint: when I did this step I had two DataFrame objects, each with ~80,000 entries in it__

In [ ]:
import pandas as pd
df = pd.read_csv('assets/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df.head()

In [ ]:
# In this code cell, transform the Data_Value column
df_C = df[:]
df_C['Data_Value'] = df_C['Data_Value']/10
print(df_C)
df_C_TMAX = df_C[df_C['Element'] == 'TMAX']
print(df_C_TMAX)
df_C_TMIN = df_C[df_C['Element'] == 'TMIN']
print(df_C_TMIN.info())

## Step 2
In order to visualize the data we would plot the min and max data for each day of the year between the years 2005 and 2014 across all weather stations. But we also need to find out when the min or max temperature in 2015 falls below the min or rises above the max for the previous decade.

If you did step 1 you have two Series objects with min and max times for the years 2005 through 2015. You can use Pandas `groupby` to create max and min temperature Series objects across all weather stations for each day of these years, and you can deal with the records for February 29 (the leap year) by dropping them.

__hint: when I finished this step, I had two DataFrame objects, each with exactly 4015 observations in them__

In [ ]:
# create a DataFrame of maximum temperature by date
df_C_TMAX_gD = pd.DataFrame(data = df_C_TMAX.groupby(['Date'])['Data_Value'].max().drop(['2008-02-29','2012-02-29'])) 
print(df_C_TMAX_gD)
# create a DataFrame of minimum temperatures by date
df_C_TMIN_gD = pd.DataFrame(data = df_C_TMIN.groupby(['Date'])['Data_Value'].min().drop(['2008-02-29','2012-02-29']))
print(df_C_TMIN_gD)

## Step 3
Now that you have grouped the daily max and min temperatures for each day of the years 2005 through 2015, you can separate out the data for 2015. Then you can use the Pandas `groupby` function to find the max and min of the temperature data for each __day of the year__ for the 2005-2014 data.

__hint: at the end of this step I had two DataFrames, one of maximum and the other of minimum values, which each had 365 observations in them. I also had another pair of similar DataFrames but only for the year 2015.__

In [ ]:
df_C_TMAX_gD.loc[:, 'Y'] = df_C_TMAX_gD.index.str.split('-').str[0]
df_C_TMAX_gD.loc[:, 'M-D'] = df_C_TMAX_gD.index.str.split('-').str[1] + '-' + df_C_TMAX_gD.index.str.split('-').str[2]
df_C_TMIN_gD.loc[:, 'Y'] = df_C_TMIN_gD.index.str.split('-').str[0]
df_C_TMIN_gD.loc[:, 'M-D'] = df_C_TMIN_gD.index.str.split('-').str[1] + '-' + df_C_TMIN_gD.index.str.split('-').str[2]

In [ ]:
print(df_C_TMAX_gD)
print(df_C_TMIN_gD)

In [ ]:
# calculate the minimum and maximum values for the day of the year for 2005 through 2014
df_C_TMAX_gD_2005_2014 = pd.DataFrame(data = df_C_TMAX_gD[:'2014-12-31'].reset_index(drop=True).groupby(['M-D'])['Data_Value'].max())['Data_Value']
print(df_C_TMAX_gD_2005_2014)
df_C_TMIN_gD_2005_2014 = pd.DataFrame(data = df_C_TMIN_gD[:'2014-12-31'].reset_index(drop=True).groupby(['M-D'])['Data_Value'].min())['Data_Value']
print(df_C_TMIN_gD_2005_2014)
# calculate the minimum and maximum values for the years 2015
df_C_TMAX_gD_2015 = df_C_TMAX_gD['2015-01-01':].set_index(['M-D'])['Data_Value']
print(df_C_TMAX_gD_2015)
df_C_TMIN_gD_2015 = df_C_TMIN_gD['2015-01-01':].set_index(['M-D'])['Data_Value']
print(df_C_TMIN_gD_2015)

## Step 4
Now it's time to plot! You need to explore matplotlib in order to plot line graphs of the min and max temperatures for the years 2005 through 2014 and to scatter plot __only__ the daily 2015 temperatures that exceeded those values.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from calendar import month_abbr

# put your plotting code here!
f, ax= plt.subplots(1, 1, figsize=(8, 7))
plt.plot(df_C_TMAX_gD_2005_2014, 'r',
         label = 'Highest temperature of 2005 - 2004',
         alpha = .5)
plt.plot(df_C_TMIN_gD_2005_2014, 'b',
         label = 'Lowest temperature of 2005 - 2004',
         alpha = .5)
plt.scatter(df_C_TMAX_gD_2015[df_C_TMAX_gD_2015 > df_C_TMAX_gD_2005_2014].index,
            df_C_TMAX_gD_2015[df_C_TMAX_gD_2015 > df_C_TMAX_gD_2005_2014],
            color = 'r',
            marker = '*',
            label='Record highest temperature of 2015')
plt.scatter(df_C_TMIN_gD_2015[df_C_TMIN_gD_2015 < df_C_TMIN_gD_2005_2014].index,
            df_C_TMIN_gD_2015[df_C_TMIN_gD_2015 < df_C_TMIN_gD_2005_2014],
            color = 'b',
            marker='*',
            label='Record lowest temperature of 2015')
plt.gca().fill_between(range(len(df_C_TMAX_gD_2005_2014)), 
                       df_C_TMIN_gD_2005_2014, df_C_TMAX_gD_2005_2014, 
                       facecolor='grey', 
                       alpha=.3);
plt.xticks(np.arange(15, 351, 30), month_abbr[1:]);
ax.set_xlabel('Month of the year')
ax.set_ylabel('Temperature ($^o C$)')
ax.set_title('Record highest and lowest temperature by day of 2005-2015')
plt.legend(loc=4, frameon=False, title='Legend');

In [ ]:
f.savefig('plot.png')